In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
from collections import deque

In [4]:
from google.colab import drive

drive.mount('/content/drive', force_remount = True)

import sys
sys.path.append('/content/drive/My Drive/kaggle/contrail_detector/')
import utils
import models
from configs import wnet_config as Config

Mounted at /content/drive


# 1. Generate Anchors

In [5]:
normalized_anchors = models.get_anchors(Config)
np.save("/content/drive/MyDrive/kaggle/contrail_detector/data/normalized_anchors.npy",normalized_anchors)

In [6]:
normalized_anchors.shape

(16368, 4)

#2. Find the Upperbound for the Number of Boxes

In [7]:
train_metadata = pd.read_json("/content/drive/MyDrive/kaggle/contrail_detector/data/train_metadata.json", dtype={'record_id': 'str'})
valid_metadata = pd.read_json("/content/drive/MyDrive/kaggle/contrail_detector/data/valid_metadata.json", dtype={'record_id': 'str'})
test_metadata = pd.read_json("/content/drive/MyDrive/kaggle/contrail_detector/data/test_metadata.json", dtype={'record_id': 'str'})

In [8]:
# from record_id's make file paths
train_metadata['image_path'] = "/content/drive/MyDrive/kaggle/contrail_detector/data/train/image/"+ train_metadata['record_id'].astype(str) + ".npy"
train_metadata['mask_path'] = "/content/drive/MyDrive/kaggle/contrail_detector/data/train/mask/"+ train_metadata['record_id'].astype(str) + ".npy"
train_metadata.head()


,record_id,projection_wkt,row_min,row_size,col_min,col_size,timestamp,contrail_exists,image_path,mask_path
0,9075952105055778186,"PROJCS[""unknown"",GEOGCS[""unknown"",DATUM[""WGS_1...",-1811231.625,-1901.528076,834079.00000,2024.375977,2019-08-28 23:20:00,0,/content/drive/MyDrive/kaggle/contrail_detecto...,/content/drive/MyDrive/kaggle/contrail_detecto...
1,8380074962411412067,"PROJCS[""unknown"",GEOGCS[""unknown"",DATUM[""WGS_1...",4696927.500,-2058.764893,592684.56250,1868.726685,2020-02-06 05:20:00,0,/content/drive/MyDrive/kaggle/contrail_detecto...,/content/drive/MyDrive/kaggle/contrail_detecto...
2,6808716999707654664,"PROJCS[""unknown"",GEOGCS[""unknown"",DATUM[""WGS_1...",4356032.000,-1996.296997,367297.18750,1928.051147,2020-03-04 21:10:00,1,/content/drive/MyDrive/kaggle/contrail_detecto...,/content/drive/MyDrive/kaggle/contrail_detecto...
3,8019228729201913883,"PROJCS[""unknown"",GEOGCS[""unknown"",DATUM[""WGS_1...",4691404.500,-2039.323608,516009.65625,1887.738647,2019-10-17 10:00:00,1,/content/drive/MyDrive/kaggle/contrail_detecto...,/content/drive/MyDrive/kaggle/contrail_detecto...
4,1264966833918307888,"PROJCS[""unknown"",GEOGCS[""unknown"",DATUM[""WGS_1...",1187218.375,-1949.495972,243261.87500,1955.620239,2019-10-14 00:00:00,0,/content/drive/MyDrive/kaggle/contrail_detecto...,/content/drive/MyDrive/kaggle/contrail_detecto...


In [9]:
len(train_metadata)


400

In [10]:
max_n_bboxes = 0
for i in range(400) :
  if train_metadata.iloc[i].contrail_exists != 0 :
    path = train_metadata.iloc[i].mask_path
    mask = np.load(str(path)).astype(np.float32)
    bboxes, _, _, _ = utils.generate_bboxes(mask, merge_fns=['i', 'i'], thicken_thr=20, thickness=15, d_thr=0.5, i_thr=0.5, pad_ratio=0)
    max_n_bboxes = max(max_n_bboxes, bboxes.shape[0])

print(max_n_bboxes)

12


In [11]:
# from record_id's make file paths
valid_metadata['image_path'] = "/content/drive/MyDrive/kaggle/contrail_detector/data/valid/image/"+ valid_metadata['record_id'].astype(str) + ".npy"
valid_metadata['mask_path'] = "/content/drive/MyDrive/kaggle/contrail_detector/data/valid/mask/"+ valid_metadata['record_id'].astype(str) + ".npy"
valid_metadata.head()

,record_id,projection_wkt,row_min,row_size,col_min,col_size,timestamp,contrail_exists,image_path,mask_path
0,306973504044411688,"PROJCS[""unknown"",GEOGCS[""unknown"",DATUM[""WGS_1...",2685685.500,-1982.050171,444149.2500,1932.289795,2019-07-04 18:40:00,0,/content/drive/MyDrive/kaggle/contrail_detecto...,/content/drive/MyDrive/kaggle/contrail_detecto...
1,3651703694266751839,"PROJCS[""unknown"",GEOGCS[""unknown"",DATUM[""WGS_1...",-1811231.625,-1901.528076,834079.0000,2024.375977,2019-09-11 06:00:00,0,/content/drive/MyDrive/kaggle/contrail_detecto...,/content/drive/MyDrive/kaggle/contrail_detecto...
2,374029324925858720,"PROJCS[""unknown"",GEOGCS[""unknown"",DATUM[""WGS_1...",3189987.250,-2009.102417,560228.6250,1910.146118,2019-12-06 08:40:00,1,/content/drive/MyDrive/kaggle/contrail_detecto...,/content/drive/MyDrive/kaggle/contrail_detecto...
3,700009070449534432,"PROJCS[""unknown"",GEOGCS[""unknown"",DATUM[""WGS_1...",3496540.250,-2020.012207,578868.6875,1901.248535,2019-04-22 16:10:00,0,/content/drive/MyDrive/kaggle/contrail_detecto...,/content/drive/MyDrive/kaggle/contrail_detecto...
4,255772129230702955,"PROJCS[""unknown"",GEOGCS[""unknown"",DATUM[""WGS_1...",5427531.000,-2129.452881,736690.4375,1799.636108,2019-06-12 17:10:00,1,/content/drive/MyDrive/kaggle/contrail_detecto...,/content/drive/MyDrive/kaggle/contrail_detecto...


In [12]:
len(valid_metadata)

100

In [13]:
max_n_bboxes = 0
for i in range(100) :
  if valid_metadata.iloc[i].contrail_exists != 0 :
    path = valid_metadata.iloc[i].mask_path
    mask = np.load(str(path)).astype(np.float32)
    bboxes, _, _, _ = utils.generate_bboxes(mask, merge_fns=['i', 'i'], thicken_thr=20, thickness=15, d_thr=0.5, i_thr=0.5, pad_ratio=0)
    max_n_bboxes = max(max_n_bboxes, bboxes.shape[0])

print(max_n_bboxes)

14


In [14]:
# from record_id's make file paths
test_metadata['image_path'] = "/content/drive/MyDrive/kaggle/contrail_detector/data/test/image/"+ test_metadata['record_id'].astype(str) + ".npy"
test_metadata['mask_path'] = "/content/drive/MyDrive/kaggle/contrail_detector/data/test/mask/"+ test_metadata['record_id'].astype(str) + ".npy"
test_metadata.head()

,record_id,projection_wkt,row_min,row_size,col_min,col_size,timestamp,contrail_exists,image_path,mask_path
0,1150176760495756312,"PROJCS[""unknown"",GEOGCS[""unknown"",DATUM[""WGS_1...",4203634.500,-2070.590576,718113.62500,1856.578491,2019-12-27 20:40:00,1,/content/drive/MyDrive/kaggle/contrail_detecto...,/content/drive/MyDrive/kaggle/contrail_detecto...
1,1648244082045800513,"PROJCS[""unknown"",GEOGCS[""unknown"",DATUM[""WGS_1...",4207421.500,-2080.893066,763713.81250,1847.207520,2019-10-10 04:40:00,1,/content/drive/MyDrive/kaggle/contrail_detecto...,/content/drive/MyDrive/kaggle/contrail_detecto...
2,6416847877511226615,"PROJCS[""unknown"",GEOGCS[""unknown"",DATUM[""WGS_1...",1187218.375,-1949.495972,243261.87500,1955.620239,2019-10-16 20:40:00,0,/content/drive/MyDrive/kaggle/contrail_detecto...,/content/drive/MyDrive/kaggle/contrail_detecto...
3,9031747867509544812,"PROJCS[""unknown"",GEOGCS[""unknown"",DATUM[""WGS_1...",2694049.500,-2011.715088,664723.43750,1907.868774,2020-02-10 00:00:00,0,/content/drive/MyDrive/kaggle/contrail_detecto...,/content/drive/MyDrive/kaggle/contrail_detecto...
4,6100739243303189316,"PROJCS[""unknown"",GEOGCS[""unknown"",DATUM[""WGS_1...",2187728.000,-1978.681396,488176.90625,1933.758911,2019-10-16 02:00:00,0,/content/drive/MyDrive/kaggle/contrail_detecto...,/content/drive/MyDrive/kaggle/contrail_detecto...


In [15]:
len(test_metadata)

100

In [16]:
max_n_bboxes = 0
for i in range(100) :
  if test_metadata.iloc[i].contrail_exists != 0 :
    path = test_metadata.iloc[i].mask_path
    mask = np.load(str(path)).astype(np.float32)
    bboxes, _, _, _ = utils.generate_bboxes(mask, merge_fns=['i', 'i'], thicken_thr=20, thickness=15, d_thr=0.5, i_thr=0.5, pad_ratio=0)
    max_n_bboxes = max(max_n_bboxes, bboxes.shape[0])

print(max_n_bboxes)

20


# 3. Generate gt_boxes and gt_class_ids

In [17]:
for i in range(400) :
  if train_metadata.iloc[i].contrail_exists != 0 :
    path = train_metadata.iloc[i].mask_path
    mask = np.load(str(path)).astype(np.float32)
    bboxes, _, _, _ = utils.generate_bboxes(mask, merge_fns=['i', 'i'], thicken_thr=20, thickness=15, d_thr=0.5, i_thr=0.5, pad_ratio=0)
    class_ids = np.ones(bboxes.shape[0], dtype=np.int32)
    if bboxes.shape[0] > Config.max_gt_instances :
      bboxes = bboxes[-Config.max_gt_instances:]
      class_ids = class_ids[-Config.max_gt_instances:]
    elif bboxes.shape[0] < Config.max_gt_instances :
      bboxes = np.pad(bboxes, pad_width=((0, Config.max_gt_instances-bboxes.shape[0]), (0, 0)), mode='constant', constant_values=0)
      class_ids = np.pad(class_ids, pad_width=((0, Config.max_gt_instances-class_ids.shape[0]),), mode='constant', constant_values=0)
  else :
    bboxes = np.zeros((Config.max_gt_instances,4))
    class_ids = np.zeros(Config.max_gt_instances)
  np.save("/content/drive/MyDrive/kaggle/contrail_detector/data/train/gt_bboxes/" + str(train_metadata.iloc[i].record_id) + ".npy", bboxes)
  np.save("/content/drive/MyDrive/kaggle/contrail_detector/data/train/gt_class_ids/" + str(train_metadata.iloc[i].record_id) + ".npy", class_ids)

In [18]:
# normalized_anchors =
for i in range(100) :
  if valid_metadata.iloc[i].contrail_exists != 0 :
    path = valid_metadata.iloc[i].mask_path
    mask = np.load(str(path)).astype(np.float32)
    bboxes, _, _, _ = utils.generate_bboxes(mask, merge_fns=['i', 'i'], thicken_thr=20, thickness=15, d_thr=0.5, i_thr=0.5, pad_ratio=0)
    class_ids = np.ones(bboxes.shape[0], dtype=np.int32)
    if bboxes.shape[0] > Config.max_gt_instances :
      bboxes = bboxes[-Config.max_gt_instances:]
      class_ids = class_ids[-Config.max_gt_instances:]
    elif bboxes.shape[0] < Config.max_gt_instances :
      bboxes = np.pad(bboxes, pad_width=((0, Config.max_gt_instances-bboxes.shape[0]), (0, 0)), mode='constant', constant_values=0)
      class_ids = np.pad(class_ids, pad_width=((0, Config.max_gt_instances-class_ids.shape[0]),), mode='constant', constant_values=0)
  else :
    bboxes = np.zeros((Config.max_gt_instances,4))
    class_ids = np.zeros(Config.max_gt_instances)
  np.save("/content/drive/MyDrive/kaggle/contrail_detector/data/valid/gt_bboxes/" + str(valid_metadata.iloc[i].record_id) + ".npy", bboxes)
  np.save("/content/drive/MyDrive/kaggle/contrail_detector/data/valid/gt_class_ids/" + str(valid_metadata.iloc[i].record_id) + ".npy", class_ids)

In [19]:
for i in range(100) :
  if test_metadata.iloc[i].contrail_exists != 0 :
    path = test_metadata.iloc[i].mask_path
    mask = np.load(str(path)).astype(np.float32)
    bboxes, _, _, _ = utils.generate_bboxes(mask, merge_fns=['i', 'i'], thicken_thr=20, thickness=15, d_thr=0.5, i_thr=0.5, pad_ratio=0)
    class_ids = np.ones(bboxes.shape[0], dtype=np.int32)
    if bboxes.shape[0] > Config.max_gt_instances :
      bboxes = bboxes[-Config.max_gt_instances:]
      class_ids = class_ids[-Config.max_gt_instances:]
    elif bboxes.shape[0] < Config.max_gt_instances :
      bboxes = np.pad(bboxes, pad_width=((0, Config.max_gt_instances-bboxes.shape[0]), (0, 0)), mode='constant', constant_values=0)
      class_ids = np.pad(class_ids, pad_width=((0, Config.max_gt_instances-class_ids.shape[0]),), mode='constant', constant_values=0)
  else :
    bboxes = np.zeros((Config.max_gt_instances,4))
    class_ids = np.zeros(Config.max_gt_instances)
  np.save("/content/drive/MyDrive/kaggle/contrail_detector/data/test/gt_bboxes/" + str(test_metadata.iloc[i].record_id) + ".npy", bboxes)
  np.save("/content/drive/MyDrive/kaggle/contrail_detector/data/test/gt_class_ids/" + str(test_metadata.iloc[i].record_id) + ".npy", class_ids)

# 4. Generate rpn_match and rpn_boxes

In [20]:
normalized_anchors = np.load("/content/drive/MyDrive/kaggle/contrail_detector/data/normalized_anchors.npy")

In [24]:
for i in range(400) :
  if train_metadata.iloc[i].contrail_exists != 0 :
    # (anchors, gt_class_ids, gt_boxes, Config)
    gt_class_ids = np.load("/content/drive/MyDrive/kaggle/contrail_detector/data/train/gt_class_ids/" + str(train_metadata.iloc[i].record_id) + ".npy")
    gt_bboxes = np.load("/content/drive/MyDrive/kaggle/contrail_detector/data/train/gt_bboxes/" + str(train_metadata.iloc[i].record_id) + ".npy")
    rpn_matches, rpn_deltas= models.build_rpn_targets(normalized_anchors, gt_bboxes, Config)
  else :
    rpn_matches = np.full((normalized_anchors.shape[0],), -1, dtype=np.int32)
    rpn_deltas = np.zeros((Config.rpn_train_anchors_per_image, 4))
  np.save("/content/drive/MyDrive/kaggle/contrail_detector/data/train/rpn_matches/" + str(train_metadata.iloc[i].record_id) + ".npy", rpn_matches)
  np.save("/content/drive/MyDrive/kaggle/contrail_detector/data/train/rpn_deltas/" + str(train_metadata.iloc[i].record_id) + ".npy", rpn_deltas)

In [26]:
for i in range(100) :
  if valid_metadata.iloc[i].contrail_exists != 0 :
    # (anchors, gt_class_ids, gt_boxes, config)
    gt_class_ids = np.load("/content/drive/MyDrive/kaggle/contrail_detector/data/valid/gt_class_ids/" + str(valid_metadata.iloc[i].record_id) + ".npy")
    gt_bboxes = np.load("/content/drive/MyDrive/kaggle/contrail_detector/data/valid/gt_bboxes/" + str(valid_metadata.iloc[i].record_id) + ".npy")
    rpn_matches, rpn_deltas= models.build_rpn_targets(normalized_anchors, gt_bboxes, Config)
  else :
    rpn_matches = np.full((normalized_anchors.shape[0],), -1, dtype=np.int32)
    rpn_deltas = np.zeros((Config.rpn_train_anchors_per_image, 4))
  np.save("/content/drive/MyDrive/kaggle/contrail_detector/data/valid/rpn_matches/" + str(valid_metadata.iloc[i].record_id) + ".npy", rpn_matches)
  np.save("/content/drive/MyDrive/kaggle/contrail_detector/data/valid/rpn_deltas/" + str(valid_metadata.iloc[i].record_id) + ".npy", rpn_deltas)

In [27]:
for i in range(100) :
  if test_metadata.iloc[i].contrail_exists != 0 :
    # (anchors, gt_class_ids, gt_boxes, Config)
    gt_class_ids = np.load("/content/drive/MyDrive/kaggle/contrail_detector/data/test/gt_class_ids/" + str(test_metadata.iloc[i].record_id) + ".npy")
    gt_bboxes = np.load("/content/drive/MyDrive/kaggle/contrail_detector/data/test/gt_bboxes/" + str(test_metadata.iloc[i].record_id) + ".npy")
    rpn_matches, rpn_deltas= models.build_rpn_targets(normalized_anchors, gt_bboxes, Config)
  else :
    rpn_matches = np.full((normalized_anchors.shape[0],), -1, dtype=np.int32)
    rpn_deltas = np.zeros((Config.rpn_train_anchors_per_image, 4))
  np.save("/content/drive/MyDrive/kaggle/contrail_detector/data/test/rpn_matches/" + str(test_metadata.iloc[i].record_id) + ".npy", rpn_matches)
  np.save("/content/drive/MyDrive/kaggle/contrail_detector/data/test/rpn_deltas/" + str(test_metadata.iloc[i].record_id) + ".npy", rpn_deltas)

# 5. Generate masked_images

In [75]:
for i in range(400):
    image_path = train_metadata.iloc[i].image_path
    image = np.load(str(image_path)).astype(np.float32)  # shape = (256, 256, 8, 3)

    gt_bboxes_path = "/content/drive/MyDrive/kaggle/contrail_detector/data/train/gt_bboxes/" + str(train_metadata.iloc[i].record_id) + ".npy"
    gt_bboxes = np.load(gt_bboxes_path)/256

    gt_attention_mask = utils.make_attention_mask(Config.image_shape, gt_bboxes)


    gt_attention_mask = gt_attention_mask.numpy()  # Convert the tensor to a numpy array

    gt_attention_mask = np.expand_dims(np.expand_dims(gt_attention_mask, axis=-1), axis=-1)  # Add two new axes at the end

    image = image * gt_attention_mask

    save_path = "/content/drive/MyDrive/kaggle/contrail_detector/data/train/masked_image/" + str(train_metadata.iloc[i].record_id) + ".npy"
    np.save(save_path, image)



In [76]:
for i in range(100) :
  image_path = valid_metadata.iloc[i].image_path
  image = np.load(str(image_path)).astype(np.float32)  # shape = (256, 256, 8, 3)

  gt_bboxes_path = "/content/drive/MyDrive/kaggle/contrail_detector/data/valid/gt_bboxes/" + str(valid_metadata.iloc[i].record_id) + ".npy"
  gt_bboxes = np.load(gt_bboxes_path)/256

  gt_attention_mask = utils.make_attention_mask(Config.image_shape, gt_bboxes)


  gt_attention_mask = gt_attention_mask.numpy()  # Convert the tensor to a numpy array

  gt_attention_mask = np.expand_dims(np.expand_dims(gt_attention_mask, axis=-1), axis=-1)  # Add two new axes at the end

  image = image * gt_attention_mask

  save_path = "/content/drive/MyDrive/kaggle/contrail_detector/data/valid/masked_image/" + str(valid_metadata.iloc[i].record_id) + ".npy"
  np.save(save_path, image)

In [77]:
for i in range(100) :
  image_path = test_metadata.iloc[i].image_path
  image = np.load(str(image_path)).astype(np.float32)  # shape = (256, 256, 8, 3)

  gt_bboxes_path = "/content/drive/MyDrive/kaggle/contrail_detector/data/test/gt_bboxes/" + str(test_metadata.iloc[i].record_id) + ".npy"
  gt_bboxes = np.load(gt_bboxes_path)/256

  gt_attention_mask = utils.make_attention_mask(Config.image_shape, gt_bboxes)


  gt_attention_mask = gt_attention_mask.numpy()  # Convert the tensor to a numpy array

  gt_attention_mask = np.expand_dims(np.expand_dims(gt_attention_mask, axis=-1), axis=-1)  # Add two new axes at the end

  image = image * gt_attention_mask

  save_path = "/content/drive/MyDrive/kaggle/contrail_detector/data/test/masked_image/" + str(test_metadata.iloc[i].record_id) + ".npy"
  np.save(save_path, image)

# 6. Log Datasets to W&B

In [79]:
!pip install wandb
import wandb
!wandb login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.0 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=76dbba406fb0d1ba0131cfa1141a0a24d2949aa5714a90e2d13f10ece67acdb7
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [80]:
# Initialize W&B run
run = wandb.init(project='contrail_detector')

# Creating an artifact
artifact = wandb.Artifact('my_dataset', type='dataset')

# Adding a parent directory with multiple subdirectories to the artifact
artifact.add_dir('/content/drive/MyDrive/kaggle/contrail_detector/data/')

# Logging the artifact
run.log_artifact(artifact)

wandb.finish()


wandb: Currently logged in as: jun23ird. Use `wandb login --relogin` to force relogin


wandb: Adding directory to artifact (/content/drive/MyDrive/kaggle/contrail_detector/data)... Done. 75.9s
